## Election Data Scrape Wikipedia

## Running with Classes

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # Ensure Python can find your modules

from scrapers.aus_2025 import Aus2025Scraper

scraper = Aus2025Scraper()
data = scraper.extract_tables()

print(f"Extracted {len(data)} tables.")

for i, df in enumerate(data):
    print(f"\nPolling Table {i}:")
    print(df.head())

Extracted 4 tables.

Polling Table 0:
  ('Date', 'Date')                                 ('Brand', 'Brand')  \
0      28 Mar 2025  The 2025 Australian federal election is called...   
1   13–24 Mar 2025                                   Redbridge/Accent   
2   17–23 Mar 2025                                         Roy Morgan   
3   14–19 Mar 2025                                             YouGov   
4   12–16 Mar 2025                                          Essential   

                ('Interview mode', 'Interview mode')  \
0  The 2025 Australian federal election is called...   
1                                             Online   
2                                             Online   
3                                             Online   
4                                             Online   

                      ('Sample size', 'Sample size')  \
0  The 2025 Australian federal election is called...   
1                                               2039   
2                 

c:\Users\zuc_r\Documents\repositories\electiondata\scrapers\aus_2025.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dataframes = [pd.read_html(str(table))[0] for table in polling_tables]
c:\Users\zuc_r\Documents\repositories\electiondata\scrapers\aus_2025.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dataframes = [pd.read_html(str(table))[0] for table in polling_tables]
c:\Users\zuc_r\Documents\repositories\electiondata\scrapers\aus_2025.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  all_dataframes = [pd.read_html(str(table))[0] for table in polling_tables]
c:\Users\zuc_r\Documents\repositor

## Running in Notebook

### 2025

In [31]:
!openssl genrsa -out rsa_key.pem 2048
!openssl rsa -in rsa_key.pem -pubout -out rsa_key.pub

'openssl' is not recognized as an internal or external command,
operable program or batch file.
'openssl' is not recognized as an internal or external command,
operable program or batch file.


In [77]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import re
from datetime import datetime

# Suppress FutureWarnings from pandas
warnings.simplefilter(action="ignore", category=FutureWarning)

# Wikipedia URL
url = "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2025_Australian_federal_election"

# Request the page
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Define possible top and bottom headers (case-insensitive search)
top_headers = ["Voting Intention", "National Polling"]
bottom_headers = ["Preferred Prime Minister", "Leadership Polling"]

# Find all header tags (h2, h3, h4)
headers = soup.find_all(["h2", "h3", "h4"])

# Locate polling tables
polling_tables = []
found_top = False
for header in headers:
    header_text = header.get_text(strip=True).lower()
    if any(top.lower() in header_text for top in top_headers):
        found_top = True
        continue
    if found_top and any(bottom.lower() in header_text for bottom in bottom_headers):
        break  
    if found_top:
        table = header.find_next("table", class_="wikitable")
        if table:
            polling_tables.append(table)

# Convert tables to DataFrames
all_dataframes = [pd.read_html(str(table))[0] for table in polling_tables]

# Function to remove footnotes (bracketed text)
def remove_footnotes(text):
    """Remove bracketed text and surrounding brackets from strings."""
    return re.sub(r"\[.*?\]", "", str(text)).strip()

# Clean column headers and remove footnotes across all DataFrames
for i, df in enumerate(all_dataframes):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            col1 if col1 == col2 else f"{col1}: {col2}"
            for col1, col2 in zip(df.columns.get_level_values(0), df.columns.get_level_values(1))
        ]
        df = df.iloc[0:].reset_index(drop=True)
        all_dataframes[i] = df  

    df.columns = [remove_footnotes(col) for col in df.columns]
    all_dataframes[i] = df.applymap(remove_footnotes)

# Function to convert percentages to decimals and handle missing values
def percentage_to_decimal(value):
    """Convert percentage values to decimals and replace dashes with 0."""
    if isinstance(value, str):
        value = value.strip()
        if "%" in value:
            try:
                return round(float(value.replace("%", "")) / 100, 4)
            except ValueError:
                return 0.0
        elif value in ["—", "-", ""]:
            return 0.0
    return value

# Extract Mapping Table (Events During Polling Period)
event_mapping = []

def is_event_text(value):
    """Determine if a value is an event description rather than a percentage or missing data."""
    if isinstance(value, str):
        return not re.match(r"^\d+(\.\d+)?%?$", value) and value.strip() not in ["—", "-"]
    return False

## figures out the date range from the date strings
def parse_date_range(date_str):
    """Extract start and end dates from date strings."""
    date_str = date_str.replace("–", "-").replace("—", "-")
    date_parts = date_str.split("-")
    try:
        if len(date_parts) == 2:  
            year = date_parts[1].strip()[-4:]
            if any(char.isalpha() for char in date_parts[0]):
                start_date = datetime.strptime(date_parts[0].strip() + " " + year, "%d %b %Y").strftime("%Y-%m-%d")
            else:
                month_year = " ".join(date_parts[1].strip().split()[-2:])
                start_date = datetime.strptime(date_parts[0].strip() + " " + month_year, "%d %b %Y").strftime("%Y-%m-%d")
            end_date = datetime.strptime(date_parts[1].strip(), "%d %b %Y").strftime("%Y-%m-%d")
        else:  
            start_date = end_date = datetime.strptime(date_str.strip(), "%d %b %Y").strftime("%Y-%m-%d")
        return start_date, end_date
    except ValueError:
        return None, None  

for i, df in enumerate(all_dataframes):
    if any("Primary vote" in col for col in df.columns):
        rows_to_remove = []
        for index, row in df.iterrows():
            if any(is_event_text(row[col]) for col in df.columns if "Primary vote" in col):
                start_date, end_date = parse_date_range(row[0])  
                event_mapping.append({
                    "Start Date": start_date,
                    "End Date": end_date,
                    "Event": next(row[col] for col in df.columns if "Primary vote" in col and is_event_text(row[col]))
                })
                rows_to_remove.append(index)
        all_dataframes[i] = df.drop(rows_to_remove).reset_index(drop=True)

# Convert to DataFrame
event_mapping_df = pd.DataFrame(event_mapping)

# Clean and format data in polling tables
for i, df in enumerate(all_dataframes):
    if df.shape[0] > 0 and df.shape[1] > 0:  
        start_dates = []
        end_dates = []
        for date in df.iloc[:, 0]:  
            start_date, end_date = parse_date_range(str(date))
            start_dates.append(start_date)
            end_dates.append(end_date)
        
        df.insert(1, "Start Date", start_dates)
        df.insert(2, "End Date", end_dates)

        df = df.applymap(percentage_to_decimal)

        all_dataframes[i] = df  

# Debugging Output
print(f"Extracted {len(all_dataframes)} national polling tables.")
# Print the shape of each individual dataframe
for i, df in enumerate(all_dataframes):
    print(f"Shape of polling table {i}: {df.shape}")

# Event Mapping Final Table
eventmapping2025 = event_mapping_df
print("\nEvent Mapping Table:")
print(event_mapping_df)

# Polling Data Final - Merge all extracted polling tables into a single DataFrame
pollingdata2025 = pd.concat(all_dataframes, ignore_index=True, sort=False)
print(f"\nFinal Merged Polling Data Shape: {pollingdata2025.shape}")
display(pollingdata2025.head())

Extracted 4 national polling tables.
Shape of polling table 0: (41, 15)
Shape of polling table 1: (132, 15)
Shape of polling table 2: (74, 15)
Shape of polling table 3: (15, 15)

Event Mapping Table:
   Start Date    End Date                                              Event
0  2025-03-28  2025-03-28  The 2025 Australian federal election is called...
1  2024-04-13  2024-04-13  The Liberals are re-elected in the 2024 Cook b...
2  2024-03-02  2024-03-02  Labor is re-elected in the 2024 Dunkley by-ele...
3  2023-10-14  2023-10-14  The 2023 Australian Indigenous Voice referendu...
4  2023-07-15  2023-07-15   LNP is re-elected in the 2023 Fadden by-election
5  2023-04-01  2023-04-01              Labor wins the 2023 Aston by-election
6  2022-12-23  2022-12-23  Andrew Gee leaves the Nationals to become an I...
7  2022-05-29  2022-05-29  Peter Dutton elected unopposed as Leader of th...

Final Merged Polling Data Shape: (262, 16)


,Date,Start Date,End Date,Brand,Interview mode,Sample size,Primary vote: L/NP,Primary vote: ALP,Primary vote: GRN,Primary vote: ONP,Primary vote: TOP,Primary vote: OTH,Primary vote: UND,2pp vote: ALP,2pp vote: L/NP,Primary vote: UAP
0,28–30 Mar 2025,2025-03-28,2025-03-30,Freshwater Strategy,Online,1059,0.390,0.320,0.120,0.000,0.0,0.170,0.0,0.490,0.510,NaN
1,26–30 Mar 2025,2025-03-26,2025-03-30,Resolve Strategic,Online,3237,0.370,0.290,0.130,0.070,0.0,0.140,0.0,0.500,0.500,NaN
2,27–29 Mar 2025,2025-03-27,2025-03-29,Newspoll,Online,1249,0.370,0.330,0.120,0.060,0.0,0.120,0.0,0.510,0.490,NaN
3,27 Feb – 26 Mar 2025,2025-02-27,2025-03-26,YouGov MRP,Online,10217,0.355,0.298,0.132,0.093,0.0,0.122,0.0,0.502,0.498,NaN
4,13–24 Mar 2025,2025-03-13,2025-03-24,Redbridge/Accent,Online,2039,0.380,0.340,0.110,0.000,0.0,0.170,0.0,0.510,0.490,NaN


### Connecting to Snowflake and loading

In [ ]:
import snowflake.connector
import pandas as pd
import logging
import numpy as np

# logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.WARNING)
# Path to your private key
private_key_path = "C:/Users/zuc_r/Documents/repositories/rsa_key.der"
print(private_key_path)

# Read and prepare the private key
with open(private_key_path, "rb") as key_file:
    private_key = key_file.read()
print(private_key)

# Snowflake Connection Details (replace with your credentials)
snowflake_account = "ZENMJOJ-MP21134"
snowflake_user = "SA_PYTHON"
snowflake_warehouse = "COMPUTE_WH"
snowflake_database = "ELECTION_DATA"
snowflake_schema = "PUBLIC"

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=snowflake_user,
    private_key=private_key,
    account=snowflake_account,
    database=snowflake_database,
    schema=snowflake_schema,
    warehouse=snowflake_warehouse
)

cursor = conn.cursor()
print("Successfully Connected.")

# Truncate existing tables before inserting new data
cursor.execute("TRUNCATE TABLE raw_pollingdata2025")
# cursor.execute("TRUNCATE TABLE raw_eventmapping2025")

def clean_column_name(col_name):
    """Convert column names to Snowflake-friendly format by replacing punctuation and spaces with underscores."""
    col_name = re.sub(r"[^\w\s]", "", col_name)  # Remove punctuation except underscores
    col_name = re.sub(r"\s+", "_", col_name)  # Replace spaces with underscores
    return col_name.upper()  # Snowflake prefers uppercase column names

pollingdata2025.columns = ["date", "start_date", "end_date", "brand", "interview_mode", "sample_size", "primary_vote_lnp", "primary_vote_alp", "primary_vote_grn", "primary_vote_onp", "primary_vote_top", "primary_vote_oth", "primary_vote_und", "twopp_vote_alp", "twopp_vote_lnp", "primary_vote_uap"]
pollingdata2025.to_csv("C:/Users/zuc_r/Documents/repositories/electiondata/pollingdata2025.csv")

# Function to insert data from a DataFrame into Snowflake
def insert_dataframe_into_snowflake(df, table_name):
    if df.empty:
        print(f"Skipping {table_name}, no data to insert.")
        return

    # Replace NaN values with None (which is interpreted as NULL by Snowflake)
    # Columns to apply specific changes
    # numeric_columns = ['Sample size','Primary vote: L/NP','Primary vote: ALP','Primary vote: GRN','Primary vote: ONP','Primary vote: TOP','Primary vote: OTH','Primary vote: UND','2pp vote: ALP','2pp vote: L/NP','Primary vote: UAP']
    # non_numeric_columns = ['Brand', 'Interview mode']
    # # Set NaN to 0 in numeric columns, and non-numeric values to 0
    # for col in numeric_columns:
    #     df[col] = df[col].apply(lambda x: 0 if isinstance(x, str) or pd.isna(x) else x)

    # # Set NaN to None in non-numeric columns
    # for col in non_numeric_columns:
    #     df[col] = df[col].apply(lambda x: None if pd.isna(x) else x)

    # print(df)
    # columns = "date, start_date, end_date, brand, interview_mode, sample_size, primary_vote_lnp, primary_vote_alp, primary_vote_grn, primary_vote_onp, primary_vote_top, primary_vote_oth, primary_vote_und, twopp_vote_alp, twopp_vote_lnp, primary_vote_uap"
    # print(columns)
    # values_placeholders = ", ".join(["%s"] * len(df.columns))
    # insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_placeholders})"
    # print(insert_query)

    # data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]
    # print(data_tuples)
    # cursor.executemany(insert_query, data_tuples)
    # conn.commit()

# Insert polling data
# insert_dataframe_into_snowflake(pollingdata2025, "raw_pollingdata2025")

# Insert event mapping data
# insert_dataframe_into_snowflake(eventmapping2025, "eventmapping2025")

#Close connection
cursor.close()
conn.close()

print("Data successfully uploaded to Snowflake.")

C:/Users/zuc_r/Documents/repositories/rsa_key.der
b'0\x82\x04\xbd\x02\x01\x000\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x04\x82\x04\xa70\x82\x04\xa3\x02\x01\x00\x02\x82\x01\x01\x00\xd6\t\xca!h\xca\xa3\x91\x83\x93*\x038\xf7\x88f\xa0\x96b\xe8\x91\x12\x13\t3\xfa\xac\x91\n,G\xc5(\xad\xf1\x19d\xe9SI\xdaR\xe4\x1e"\xe5d\x05\xe7\xe4\xee\x90F\xedF\xc5\x88\x0c]i\xf1\xc9\x0b&\xc2\x02\x9fE\x19\x81\xa6\xfc\xfch+m;=a\xd2\xae\xef\xd8W\xa6G{\xbbR\xea\\>\xf8v]\xfa\x06+($\xfeB\x830\x13U\xffYh\xc1\xf6o\xc9\xaf\x88\xff:D\xf5\xb5\x98T\x0c!\xac\x15\x93q\x98\xc3\xa8\xe0 \xadN\x0e{\x90 \xe6\xca\x9c8\xe0\x98\xc7\xdfp \xf9D)\x13c\xb3=q\xfeE\x0f\xbf\xcf\x94z\x85e\xf5\xcc\xc1\x01\xa7\xeb\xc6\xac\x9c6\x15\xb6\xf1\x18\x90)\xf6\x89t\xff\xf7\x81\x01\x88\xa3n\xea\xe1N\xef\t\xa1\xe9\xbc\x87\xb2\xf2\xda\xc8u\x997\xfcT\x19j\x81G\xd2\x04\xa4y8wd)m^\x1fl\r\x97\xb9\xfdr\x94\x93\x9a\xe1\xac\xf9\xc9"61f\x96\x80\x7f\x0f\x89R?\xa9\xdb\xe8\x1b\x1c:}\x02\x03\x01\x00\x01\x02\x82\x01\x00_\xc4\x9e\x93\xe7J\xef\xf7\xfej\xbc\xfc\x1

### 2022

In [83]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import re
from datetime import datetime

# Suppress FutureWarnings from pandas
warnings.simplefilter(action="ignore", category=FutureWarning)

# Wikipedia URL
url = "https://en.wikipedia.org/wiki/Opinion_polling_for_the_2022_Australian_federal_election"

# Request the page
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Define possible top and bottom headers (case-insensitive search)
top_headers = ["Voting Intention", "National Polling"]
bottom_headers = ["Preferred Prime Minister", "Leadership Polling","Polling for individual seats"]

# Find all header tags (h2, h3, h4)
headers = soup.find_all(["h2", "h3", "h4"])

# Locate polling tables
polling_tables = []
found_top = False
for header in headers:
    header_text = header.get_text(strip=True).lower()
    if any(top.lower() in header_text for top in top_headers):
        found_top = True
        continue
    if found_top and any(bottom.lower() in header_text for bottom in bottom_headers):
        break  
    if found_top:
        table = header.find_next("table", class_="wikitable")
        if table:
            polling_tables.append(table)

# Convert tables to DataFrames
all_dataframes = [pd.read_html(str(table))[0] for table in polling_tables]

# Function to remove footnotes (bracketed text)
def remove_footnotes(text):
    """Remove bracketed text and surrounding brackets from strings."""
    return re.sub(r"\[.*?\]", "", str(text)).strip()

# Clean column headers and remove footnotes across all DataFrames
for i, df in enumerate(all_dataframes):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            col1 if col1 == col2 else f"{col1}: {col2}"
            for col1, col2 in zip(df.columns.get_level_values(0), df.columns.get_level_values(1))
        ]
        df = df.iloc[0:].reset_index(drop=True)
        all_dataframes[i] = df  

    df.columns = [remove_footnotes(col) for col in df.columns]
    all_dataframes[i] = df.applymap(remove_footnotes)

# Function to convert percentages to decimals and handle missing values
def percentage_to_decimal(value):
    """Convert percentage values to decimals and replace dashes with 0."""
    if isinstance(value, str):
        value = value.strip()
        if "%" in value:
            try:
                return round(float(value.replace("%", "")) / 100, 4)
            except ValueError:
                return 0.0
        elif value in ["—", "-", ""]:
            return 0.0
    return value

# Extract Mapping Table (Events During Polling Period)
event_mapping = []

def is_event_text(value):
    """Determine if a value is an event description rather than a percentage or missing data."""
    if isinstance(value, str):
        return not re.match(r"^\d+(\.\d+)?%?$", value) and value.strip() not in ["—", "-"]
    return False

## figures out the date range from the date strings
def parse_date_range(date_str):
    """Extract start and end dates from date strings."""
    date_str = date_str.replace("–", "-").replace("—", "-")
    date_parts = date_str.split("-")
    try:
        if len(date_parts) == 2:  
            year = date_parts[1].strip()[-4:]
            if any(char.isalpha() for char in date_parts[0]):
                start_date = datetime.strptime(date_parts[0].strip() + " " + year, "%d %b %Y").strftime("%Y-%m-%d")
            else:
                month_year = " ".join(date_parts[1].strip().split()[-2:])
                start_date = datetime.strptime(date_parts[0].strip() + " " + month_year, "%d %b %Y").strftime("%Y-%m-%d")
            end_date = datetime.strptime(date_parts[1].strip(), "%d %b %Y").strftime("%Y-%m-%d")
        else:  
            start_date = end_date = datetime.strptime(date_str.strip(), "%d %b %Y").strftime("%Y-%m-%d")
        return start_date, end_date
    except ValueError:
        return None, None  

for i, df in enumerate(all_dataframes):
    if any("Primary vote" in col for col in df.columns):
        rows_to_remove = []
        for index, row in df.iterrows():
            if any(is_event_text(row[col]) for col in df.columns if "Primary vote" in col):
                start_date, end_date = parse_date_range(row[0])  
                event_mapping.append({
                    "Start Date": start_date,
                    "End Date": end_date,
                    "Event": next(row[col] for col in df.columns if "Primary vote" in col and is_event_text(row[col]))
                })
                rows_to_remove.append(index)
        all_dataframes[i] = df.drop(rows_to_remove).reset_index(drop=True)

# Convert to DataFrame
event_mapping_df = pd.DataFrame(event_mapping)

# Clean and format data in polling tables
for i, df in enumerate(all_dataframes):
    if df.shape[0] > 0 and df.shape[1] > 0:  
        start_dates = []
        end_dates = []
        for date in df.iloc[:, 0]:  
            start_date, end_date = parse_date_range(str(date))
            start_dates.append(start_date)
            end_dates.append(end_date)
        
        df.insert(1, "Start Date", start_dates)
        df.insert(2, "End Date", end_dates)

        df = df.applymap(percentage_to_decimal)

        all_dataframes[i] = df  

# Debugging Output
print(f"Extracted {len(all_dataframes)} national polling tables.")
# Print the shape of each individual dataframe
for i, df in enumerate(all_dataframes):
    print(f"Shape of polling table {i}: {df.shape}")

# Event Mapping Final Table
eventmapping2025 = event_mapping_df
print("\nEvent Mapping Table:")
print(event_mapping_df)

# Polling Data Final - Merge all extracted polling tables into a single DataFrame
pollingdata2025 = pd.concat(all_dataframes, ignore_index=True, sort=False)
print(f"\nFinal Merged Polling Data Shape: {pollingdata2025.shape}")
display(pollingdata2025.head())

Extracted 3 national polling tables.
Shape of polling table 0: (49, 15)
Shape of polling table 1: (49, 15)
Shape of polling table 2: (113, 14)

Event Mapping Table:
   Start Date    End Date                                              Event
0  2022-05-09  2022-05-09                                Early voting begins
1  2022-04-10  2022-04-10  Australian federal election campaign begins wi...
2  2022-05-09  2022-05-09                                Early voting begins
3  2022-04-10  2022-04-10  Australian federal election campaign begins wi...
4  2021-06-22  2021-06-22  Barnaby Joyce replaces Michael McCormack as Na...
5  2020-02-27  2020-02-27    COVID-19 pandemic declared a national emergency
6  2020-02-04  2020-02-04  Adam Bandt replaces Richard Di Natale as Green...
7  2019-05-30  2019-05-30  Anthony Albanese replaces Bill Shorten as Labo...

Final Merged Polling Data Shape: (211, 15)


,Date,Start Date,End Date,Brand,Interview mode,Sample size,Primary vote: L/NP,Primary vote: ALP,Primary vote: GRN,Primary vote: ONP,Primary vote: UAP,Primary vote: OTH,Primary vote: UND,2pp vote: L/NP,2pp vote: ALP
0,21 May 2022,2022-05-21,2022-05-21,Election,Election,Election,0.357,0.326,0.122,0.05,0.041,0.104,0.0,0.479,0.521
1,23 May–5 Jun 2022,2022-05-23,2022-06-05,ANU,CATI/online,3556,0.319,0.354,0.198,0.00,0.000,0.129,0.0,0.000,0.000
2,21–25 May 2022,2022-05-21,2022-05-25,Dynata (Exit Poll),Online,1424,0.330,0.410,0.110,0.04,0.000,0.110,0.0,0.000,0.000
3,13–19 May 2022,2022-05-13,2022-05-19,Newspoll-YouGov,Online,2188,0.350,0.360,0.120,0.05,0.030,0.090,0.0,0.470,0.530
4,15–18 May 2022,2022-05-15,2022-05-18,Ipsos,Telephone/online,1996,0.350,0.360,0.130,0.05,0.030,0.080,0.0,0.470,0.530


In [84]:
pollingdata2025.columns = ["date", "start_date", "end_date", "brand", "interview_mode", "sample_size", "primary_vote_lnp", "primary_vote_alp", "primary_vote_grn", "primary_vote_onp", "primary_vote_uap", "primary_vote_oth", "primary_vote_und", "twopp_vote_alp", "twopp_vote_lnp"]
pollingdata2025.to_csv("C:/Users/zuc_r/Documents/repositories/electiondata/pollingdata2022.csv")